# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


# Intergrax — Engine Planner (Planner-only Test Notebook)

## Purpose

This notebook is dedicated to **isolated testing and iterative improvement of the Intergrax `EnginePlanner`**, completely **independent from the engine execution pipeline** (`RuntimeEngine.run()`).

It is designed to:

- run `EnginePlanner.plan(...)` for various user prompts,
- inspect and compare generated `EnginePlan` outputs,
- evaluate plan stability, correctness, and feasibility,
- iteratively refine the planner prompt, plan schema, and validation rules,
- improve the planner safely without impacting the runtime `run()` behavior.

## Scope

This notebook:

- does **not** call `RuntimeEngine.run()`,
- does **not** execute RAG / tools / websearch / attachments retrieval,
- does **not** generate a final user answer,
- focuses exclusively on **plan generation and analysis**.

## Architectural assumptions

- `EnginePlanner` is part of the runtime, but it must be testable in isolation.
- Planning is **LLM-driven** (no decision heuristics in code).
- Availability of mechanisms (RAG, websearch, tools, attachments, user LTM) is provided explicitly as **capabilities**.
- `EnginePlan` is a strongly-typed artifact that will later drive execution in a **Planner → Executor → Verifier** architecture.

## How to use

1. Initialize the same LLM adapter and `RuntimeConfig` you use in the runtime.
2. Create a minimal `RuntimeState` containing the fields required by the planner (`base_history`, `final_system_instructions`, `cap_*`).
3. Run the planner for different prompts.
4. Inspect the resulting `EnginePlan` and its `debug` field.
5. Iterate until plans are stable, consistent, and feasible.

This notebook is the first step toward converting the engine into a true **ChatGPT-like** runtime:
**Planner → Executor → Verifier**.


In [40]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [41]:
from intergrax.llm.messages import ChatMessage
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.nexus.config import RuntimeConfig
from intergrax.runtime.nexus.planning.engine_planner import EnginePlanner

# -------------------------------------------------------
# Global test constants
# -------------------------------------------------------

USER_ID = "demo-user-planner"
SESSION_ID = "sess_planner_only_001"

# -------------------------------------------------------
# Build LLM adapter and RuntimeConfig
# IMPORTANT: Replace build_llm_adapter() with your actual builder.
# This must be the same adapter/config used in your runtime.
# -------------------------------------------------------

llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=True,
    enable_websearch=True,
    tools_mode="auto",
    enable_user_longterm_memory=True,    
)

planner = EnginePlanner(llm_adapter=llm_adapter)

## Minimal RuntimeState for planner-only tests

The planner requires a minimal subset of runtime context:

- `RuntimeRequest` (user_id, session_id, message, attachments)
- `RuntimeState` fields used by the planner:
  - `base_history` (recent messages, optional)
  - `final_system_instructions` (optional)
  - capability flags: `cap_rag_available`, `cap_user_ltm_available`, `cap_attachments_available`, `cap_websearch_available`, `cap_tools_available`

In this notebook we will set capability flags explicitly,
so we can test multiple scenarios deterministically.


In [ ]:
from intergrax.runtime.nexus.planning.engine_planner import EnginePlan
from intergrax.runtime.nexus.planning.plan_builder_helper import build_plan


async def run_planner(
    *,
    message: str,
    instructions: str | None = "Use a technical, concise style. Never use emojis in code blocks or technical documentation.",
) -> EnginePlan:
    
    plan = await build_plan(
        user_id=USER_ID,
        session_id=SESSION_ID,
        message=message,
        instructions=instructions,
        config=config,
    )
    print("\n=== ENGINE PLAN ===")
    plan.print_pretty()

    return plan


## Test 1 — Generic programming question (no retrieval expected)
A general Python engineering question. The planner should typically choose `intent=generic` and keep all retrieval flags off.


In [43]:
await run_planner(
    message="Explain how to implement an async retry strategy in Python for API calls."
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': True,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 292,
           'planner_raw_len': 292,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "generic",\n'
                                  '  "reasoning_summary": "Provided a general '
                                  'answer to the user\'s question.",\n'
                                  '  "ask_clarifying_question": false,\n'


EnginePlan(version='1.0', intent=<PlanIntent.GENERIC: 'generic'>, reasoning_summary="Provided a general answer to the user's question.", ask_clarifying_question=False, clarifying_question=None, use_websearch=False, use_user_longterm_memory=True, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'generic', 'reasoning_summary': "Provided a general answer to the user's question.", 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 292, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 292, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "generic",\n  "reasoning_summary": "Provided a general answer to the user\'s question.",\n  "ask_

## Test 2 — Freshness / up-to-date information (websearch expected)
A question that explicitly requests the most recent changes and dates. The planner should prefer `intent=freshness` and enable websearch when available.


In [44]:
await run_planner(
    message="What are the most recent major changes to the OpenAI Responses API and tool calling? Provide dates."
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': False,
                                'use_websearch': True,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 335,
           'planner_raw_len': 335,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "freshness",\n'
                                  '  "reasoning_summary": "Intent: freshness, '
                                  'User requested recent changes to OpenAI '
                                  'Responses API and tool calling."

EnginePlan(version='1.0', intent=<PlanIntent.FRESHNESS: 'freshness'>, reasoning_summary='Intent: freshness, User requested recent changes to OpenAI Responses API and tool calling.', ask_clarifying_question=False, clarifying_question=None, use_websearch=True, use_user_longterm_memory=False, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'freshness', 'reasoning_summary': 'Intent: freshness, User requested recent changes to OpenAI Responses API and tool calling.', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': True, 'use_user_longterm_memory': False, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 335, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': True, 'use_user_longterm_memory': False, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 335, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "freshnes

## Test 3 — Project-specific architecture question (LTM expected)
A question about your Intergrax architecture decisions. The planner should choose `intent=project_architecture` and enable user long-term memory when available.


In [45]:
await run_planner(
    message="In my Intergrax runtime, should StepPlanner be separate from EnginePlanner? Give a recommendation based on our architecture decisions."
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': True,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 298,
           'planner_raw_len': 298,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "project_architecture",\n'
                                  '  "reasoning_summary": "Based on Intergrax '
                                  'architecture decisions.",\n'
                                  '  "ask_clarifying_question": false

EnginePlan(version='1.0', intent=<PlanIntent.PROJECT_ARCHITECTURE: 'project_architecture'>, reasoning_summary='Based on Intergrax architecture decisions.', ask_clarifying_question=False, clarifying_question=None, use_websearch=False, use_user_longterm_memory=True, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'project_architecture', 'reasoning_summary': 'Based on Intergrax architecture decisions.', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 298, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 298, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "project_architecture",\n  "reasoning_summary": "Based on Intergrax arch

## Test 4 — Clarification required (clarify intent expected)
An ambiguous request with missing context. The planner should pick `intent=clarify` and provide exactly one clarifying question. All flags should be off.


In [46]:
await run_planner(
    message="Make it better."
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': False,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 299,
           'planner_raw_len': 299,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "generic",\n'
                                  '  "reasoning_summary": "General knowledge '
                                  'retrieval for query \'Make it better\'.",\n'
                                  '  "ask_clarifying_question": fa

EnginePlan(version='1.0', intent=<PlanIntent.GENERIC: 'generic'>, reasoning_summary="General knowledge retrieval for query 'Make it better'.", ask_clarifying_question=False, clarifying_question=None, use_websearch=False, use_user_longterm_memory=False, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'generic', 'reasoning_summary': "General knowledge retrieval for query 'Make it better'.", 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': False, 'use_user_longterm_memory': False, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 299, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': False, 'use_user_longterm_memory': False, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 299, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "generic",\n  "reasoning_summary": "General knowledge retrieval for query \'Make i

## Test 5 — Empty input (clarify intent expected)
No meaningful user query. The planner should pick `intent=clarify` and ask for the missing information. All flags should be off.


In [47]:
await run_planner(
    message="   "
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': True,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 295,
           'planner_raw_len': 295,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "generic",\n'
                                  '  "reasoning_summary": "High-level note: '
                                  'Applying constraints to user query.",\n'
                                  '  "ask_clarifying_question": false,\n

EnginePlan(version='1.0', intent=<PlanIntent.GENERIC: 'generic'>, reasoning_summary='High-level note: Applying constraints to user query.', ask_clarifying_question=False, clarifying_question=None, use_websearch=False, use_user_longterm_memory=True, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'generic', 'reasoning_summary': 'High-level note: Applying constraints to user query.', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 295, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 295, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "generic",\n  "reasoning_summary": "High-level note: Applying constraints to user query.",\

## Test 6 — Stable factual definition (no websearch, no LTM expected)
A non-time-sensitive general CS question. The planner should stay in `intent=generic` and keep retrieval flags off.


In [48]:
await run_planner(
    message="What is the difference between a mutex and a semaphore?"
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': True,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 322,
           'planner_raw_len': 322,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "generic",\n'
                                  '  "reasoning_summary": "Provided general '
                                  'information on the difference between a '
                                  'mutex and a semaphore.",\n'
        

EnginePlan(version='1.0', intent=<PlanIntent.GENERIC: 'generic'>, reasoning_summary='Provided general information on the difference between a mutex and a semaphore.', ask_clarifying_question=False, clarifying_question=None, use_websearch=False, use_user_longterm_memory=True, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'generic', 'reasoning_summary': 'Provided general information on the difference between a mutex and a semaphore.', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 322, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 322, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "generic",\n  "reasoning_summary": "P

## Test 7 — User preference / response style (borderline LTM)
The user asks how to format the response given their preferences. We want to observe whether the planner enables user long-term memory or treats it as generic.


In [49]:
await run_planner(
    message="Given my preferences for concise technical answers, how should you format your response?"
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': True,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 295,
           'planner_raw_len': 295,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "generic",\n'
                                  '  "reasoning_summary": "Provided a generic '
                                  'answer based on user preferences.",\n'
                                  '  "ask_clarifying_question": false,\n

EnginePlan(version='1.0', intent=<PlanIntent.GENERIC: 'generic'>, reasoning_summary='Provided a generic answer based on user preferences.', ask_clarifying_question=False, clarifying_question=None, use_websearch=False, use_user_longterm_memory=True, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'generic', 'reasoning_summary': 'Provided a generic answer based on user preferences.', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 295, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 295, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "generic",\n  "reasoning_summary": "Provided a generic answer based on user preferences.",\

## Test 8 — Continue from earlier context (project_architecture + LTM expected)
The user explicitly asks to continue work discussed earlier. This should strongly trigger project-specific intent and enable user long-term memory.


In [50]:
await run_planner(
    message="Continue the Intergrax planner refactor we discussed earlier and apply the same decisions."
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': True,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 321,
           'planner_raw_len': 321,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "generic",\n'
                                  '  "reasoning_summary": "Continuing the '
                                  'Intergrax planner refactor with previously '
                                  'discussed decisions.",\n'
         

EnginePlan(version='1.0', intent=<PlanIntent.GENERIC: 'generic'>, reasoning_summary='Continuing the Intergrax planner refactor with previously discussed decisions.', ask_clarifying_question=False, clarifying_question=None, use_websearch=False, use_user_longterm_memory=True, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'generic', 'reasoning_summary': 'Continuing the Intergrax planner refactor with previously discussed decisions.', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 321, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 321, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "generic",\n  "reasoning_summary": "Con

## Test 9 — Project continuation with explicit prior context in history (intent should become project_architecture)
We simulate a realistic chat history mentioning Intergrax planning decisions, then ask to continue. This checks whether intent classification improves when real context exists.


In [51]:
req = build_planner_request(
    user_id=USER_ID,
    session_id=SESSION_ID,
    message="Continue the Intergrax planner refactor we discussed earlier and apply the same decisions.",
    instructions="Use a technical, concise style. Never use emojis in code blocks or technical documentation.",
)

state = build_planner_state(req=req, run_id="planner-test-run-002")

# Simulate real context: Intergrax planning decisions already discussed
state.base_history = [
    ChatMessage(role="user", content="We are working on Intergrax nexus Runtime planning."),
    ChatMessage(role="assistant", content="Understood. EnginePlanner decides intent + capabilities, StepPlanner maps intent to steps."),
    ChatMessage(role="user", content="We decided StepPlanner must not use heuristics; EnginePlanner should produce intent and flags."),
    ChatMessage(role="assistant", content="Agreed. We'll canonicalize plan outputs and clamp to runtime capabilities."),
]

plan = await planner.plan(req=req, state=state, config=config, run_id=state.run_id)
print("\n=== ENGINE PLAN (Test 9) ===")
plan.print_pretty()



=== ENGINE PLAN (Test 9) ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': True,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 295,
           'planner_raw_len': 295,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "generic",\n'
                                  '  "reasoning_summary": "Applying previous '
                                  'decisions to the refactoring task.",\n'
                                  '  "ask_clarifying_question":

## Test 10 — Detailed generic engineering question (no LTM expected)
A detailed, but still general engineering question. The planner should remain `intent=generic` and keep user long-term memory off.


In [52]:
await run_planner(
    message=(
        "In Python, compare exponential backoff with jitter vs fixed backoff. "
        "When should each be used, and what are common implementation pitfalls?"
    )
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': True,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 318,
           'planner_raw_len': 318,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "generic",\n'
                                  '  "reasoning_summary": "Provided answer '
                                  'based on internal knowledge and '
                                  'understanding of the topic.",\n'
            

EnginePlan(version='1.0', intent=<PlanIntent.GENERIC: 'generic'>, reasoning_summary='Provided answer based on internal knowledge and understanding of the topic.', ask_clarifying_question=False, clarifying_question=None, use_websearch=False, use_user_longterm_memory=True, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'generic', 'reasoning_summary': 'Provided answer based on internal knowledge and understanding of the topic.', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 318, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 318, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "generic",\n  "reasoning_summary": "Provided 

## Test 11 — Explicit web lookup request (freshness + websearch expected)
The user explicitly asks to look up information online. The planner should choose `intent=freshness` and enable websearch (if available).


In [53]:
await run_planner(
    message="Search the web for the latest OpenAI Responses API changes and summarize them with dates."
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': True,
                                'use_user_longterm_memory': False,
                                'use_websearch': True,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 314,
           'planner_raw_len': 314,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "freshness",\n'
                                  '  "reasoning_summary": "User requested '
                                  'up-to-date information on OpenAI Responses '
                                  'API changes.",\n'
                

EnginePlan(version='1.0', intent=<PlanIntent.FRESHNESS: 'freshness'>, reasoning_summary='User requested up-to-date information on OpenAI Responses API changes.', ask_clarifying_question=False, clarifying_question=None, use_websearch=True, use_user_longterm_memory=False, use_rag=False, use_tools=True, debug={'raw_json': {'version': '1.0', 'intent': 'freshness', 'reasoning_summary': 'User requested up-to-date information on OpenAI Responses API changes.', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': True, 'use_user_longterm_memory': False, 'use_rag': False, 'use_tools': True}, 'planner_json_len': 314, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': True, 'use_user_longterm_memory': False, 'use_rag': False, 'use_tools': True}, 'planner_raw_len': 314, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "freshness",\n  "reasoning_summary": "User requested

## Test 12 — Explicit tool usage request (tools expected, websearch not necessarily)
The user explicitly requests tool usage, but not web browsing. We want to see whether the planner enables tools without enabling websearch.


In [54]:
await run_planner(
    message="Use tools to analyze my request and propose a step-by-step execution plan. Do not browse the web."
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': True,
                                'use_tools': True,
                                'use_user_longterm_memory': True,
                                'use_websearch': False,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 301,
           'planner_raw_len': 301,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "generic",\n'
                                  '  "reasoning_summary": "User requested '
                                  'step-by-step execution plan without '
                                  'websearch",\n'
                             

EnginePlan(version='1.0', intent=<PlanIntent.GENERIC: 'generic'>, reasoning_summary='User requested step-by-step execution plan without websearch', ask_clarifying_question=False, clarifying_question=None, use_websearch=False, use_user_longterm_memory=True, use_rag=True, use_tools=True, debug={'raw_json': {'version': '1.0', 'intent': 'generic', 'reasoning_summary': 'User requested step-by-step execution plan without websearch', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': True, 'use_tools': True}, 'planner_json_len': 301, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': False, 'use_user_longterm_memory': True, 'use_rag': True, 'use_tools': True}, 'planner_raw_len': 301, 'planner_raw_preview': '{\n  "version": "1.0",\n  "intent": "generic",\n  "reasoning_summary": "User requested step-by-step execution plan wi

## Test 13 — Freshness question without explicit 'search' wording (websearch expected, tools should remain off)
The user asks for the most recent changes with dates, but does not explicitly say "search the web". We want to confirm tools do not flip on.


In [55]:
await run_planner(
    message="What changed most recently in the OpenAI Responses API? Please include dates and official sources."
)



=== ENGINE PLAN ===
{'ask_clarifying_question': False,
 'clarifying_question': None,
 'debug': {'capability_clamp': {'rag_available': True,
                                'tools_available': True,
                                'use_rag': False,
                                'use_tools': False,
                                'use_user_longterm_memory': True,
                                'use_websearch': True,
                                'user_ltm_available': True,
                                'websearch_available': True},
           'planner_json_len': 342,
           'planner_raw_len': 342,
           'planner_raw_preview': '{\n'
                                  '  "version": "1.0",\n'
                                  '  "intent": "freshness",\n'
                                  '  "reasoning_summary": "User query requires '
                                  'up-to-date information, so websearch and '
                                  'user long-term memory are consi

EnginePlan(version='1.0', intent=<PlanIntent.FRESHNESS: 'freshness'>, reasoning_summary='User query requires up-to-date information, so websearch and user long-term memory are considered.', ask_clarifying_question=False, clarifying_question=None, use_websearch=True, use_user_longterm_memory=True, use_rag=False, use_tools=False, debug={'raw_json': {'version': '1.0', 'intent': 'freshness', 'reasoning_summary': 'User query requires up-to-date information, so websearch and user long-term memory are considered.', 'ask_clarifying_question': False, 'clarifying_question': None, 'use_websearch': True, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_json_len': 342, 'capability_clamp': {'websearch_available': True, 'user_ltm_available': True, 'rag_available': True, 'tools_available': True, 'use_websearch': True, 'use_user_longterm_memory': True, 'use_rag': False, 'use_tools': False}, 'planner_raw_len': 342, 'planner_raw_preview': '{\n  "version": "1.0",\n  "inten